In [1]:
import os
import re
import pandas as pd
import numpy as np

In [3]:
input_dir = './raw_data/'
files = os.listdir(input_dir)

data_list = []
for file in files:
    data_list.append(pd.read_excel(input_dir + file))

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: './raw_data/'

In [ ]:
data = pd.concat(data_list)

data = data.reset_index()
data = data.drop("index", axis=1)

print(len(data))

data.head(5)

52256


,描述,位置信息,区域,房屋信息,关注信息,总价,单价
0,威兰德装修套三对中庭，客户只给契税,威兰德小镇,双流,3室2厅 | 86.4平米 | 东 | 简装 | 中楼层(共26层) | 2016年建 | 塔楼,135人关注 / 6个月以前发布,91.8万,单价10625元/平米
1,房子清水套三户型方正采光好无遮挡，视野开阔！,南湖逸家二期,双流,3室1厅 | 66.12平米 | 东 | 毛坯 | 高楼层(共33层) | 2017年建 |...,40人关注 / 2个月以前发布,128.5万,单价19435元/平米
2,南湖逸家满二精装房，中间楼层，采光良好,南湖逸家二期,双流,3室1厅 | 73.59平米 | 南 | 精装 | 中楼层(共34层) | 2017年建 |...,58人关注 / 15天以前发布,153万,单价20791元/平米
3,佰客郡精装修房子配套成熟业主真心卖,佰客郡,双流,2室1厅 | 75.4平米 | 东 北 | 简装 | 中楼层(共16层) | 2011年建 ...,36人关注 / 2个月以前发布,89万,单价11804元/平米
4,加贝书香尚品 精装修 带家具家电出售,加贝书香尚品,双流,1室1厅 | 51.69平米 | 南 | 精装 | 高楼层(共15层) | 2007年建 | 板楼,38人关注 / 1个月以前发布,64.5万,单价12479元/平米


In [ ]:
nrec = data.房屋信息.map(lambda x: len(x.split('|')))
nrec.value_counts()

7    39045
6    12669
8      370
4      122
5       50
Name: 房屋信息, dtype: int64

In [ ]:
data = data[nrec == 7]
len(data)

39045

In [ ]:
data['户型'] = data.房屋信息.map(lambda x: x.split('|')[0])
data['面积'] = data.房屋信息.map(lambda x: x.split('|')[1])
data['朝向'] = data.房屋信息.map(lambda x: x.split('|')[2])
data['类型'] = data.房屋信息.map(lambda x: x.split('|')[3])
data['楼层'] = data.房屋信息.map(lambda x: x.split('|')[4])
data['建成时间'] = data.房屋信息.map(lambda x: x.split('|')[5])
data['结构'] = data.房屋信息.map(lambda x: x.split('|')[6])

data = data.drop('房屋信息', axis=1)
data.head()

,描述,位置信息,区域,关注信息,总价,单价,户型,面积,朝向,类型,楼层,建成时间,结构
0,威兰德装修套三对中庭，客户只给契税,威兰德小镇,双流,135人关注 / 6个月以前发布,91.8万,单价10625元/平米,3室2厅,86.4平米,东,简装,中楼层(共26层),2016年建,塔楼
1,房子清水套三户型方正采光好无遮挡，视野开阔！,南湖逸家二期,双流,40人关注 / 2个月以前发布,128.5万,单价19435元/平米,3室1厅,66.12平米,东,毛坯,高楼层(共33层),2017年建,板塔结合
2,南湖逸家满二精装房，中间楼层，采光良好,南湖逸家二期,双流,58人关注 / 15天以前发布,153万,单价20791元/平米,3室1厅,73.59平米,南,精装,中楼层(共34层),2017年建,板塔结合
3,佰客郡精装修房子配套成熟业主真心卖,佰客郡,双流,36人关注 / 2个月以前发布,89万,单价11804元/平米,2室1厅,75.4平米,东 北,简装,中楼层(共16层),2011年建,板楼
4,加贝书香尚品 精装修 带家具家电出售,加贝书香尚品,双流,38人关注 / 1个月以前发布,64.5万,单价12479元/平米,1室1厅,51.69平米,南,精装,高楼层(共15层),2007年建,板楼


In [ ]:
nrec = data.关注信息.map(lambda x: len(x.split('/')))
nrec.value_counts()

2    39045
Name: 关注信息, dtype: int64

In [ ]:
data['关注人数'] = data.关注信息.map(lambda x: x.split('/')[0])
data['发布时间'] = data.关注信息.map(lambda x: x.split('/')[1])

data = data.drop('关注信息', axis=1)
data.head()

,描述,位置信息,区域,总价,单价,户型,面积,朝向,类型,楼层,建成时间,结构,关注人数,发布时间
0,威兰德装修套三对中庭，客户只给契税,威兰德小镇,双流,91.8万,单价10625元/平米,3室2厅,86.4平米,东,简装,中楼层(共26层),2016年建,塔楼,135人关注,6个月以前发布
1,房子清水套三户型方正采光好无遮挡，视野开阔！,南湖逸家二期,双流,128.5万,单价19435元/平米,3室1厅,66.12平米,东,毛坯,高楼层(共33层),2017年建,板塔结合,40人关注,2个月以前发布
2,南湖逸家满二精装房，中间楼层，采光良好,南湖逸家二期,双流,153万,单价20791元/平米,3室1厅,73.59平米,南,精装,中楼层(共34层),2017年建,板塔结合,58人关注,15天以前发布
3,佰客郡精装修房子配套成熟业主真心卖,佰客郡,双流,89万,单价11804元/平米,2室1厅,75.4平米,东 北,简装,中楼层(共16层),2011年建,板楼,36人关注,2个月以前发布
4,加贝书香尚品 精装修 带家具家电出售,加贝书香尚品,双流,64.5万,单价12479元/平米,1室1厅,51.69平米,南,精装,高楼层(共15层),2007年建,板楼,38人关注,1个月以前发布


In [ ]:
data.isnull()

,描述,位置信息,区域,总价,单价,户型,面积,朝向,类型,楼层,建成时间,结构,关注人数,发布时间
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52248,False,False,False,False,False,False,False,False,False,False,False,False,False,False
52249,False,False,False,False,False,False,False,False,False,False,False,False,False,False
52250,False,False,False,False,False,False,False,False,False,False,False,False,False,False
52251,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
(data.isnull()).sum()

描述      0
位置信息    0
区域      0
总价      0
单价      0
户型      0
面积      0
朝向      0
类型      0
楼层      0
建成时间    0
结构      0
关注人数    0
发布时间    0
dtype: int64

In [ ]:
(data.duplicated()).sum()

16

In [ ]:
data.drop_duplicates(inplace=True)

In [ ]:
data.总价.map(lambda x: re.sub('[\u4E00-\u9FA5]', '', x)).unique()

array(['91.8', '128.5', '153', ..., '447', '125.5', '122.6'], dtype=object)